In [ ]:
import tarfile
import os

In [ ]:
with tarfile.open('build_cache/nocereal/tuplex.tar', 'r') as tf:
    mems = tf.getmembers()

In [ ]:
mems[1].get_info()['name']

In [ ]:
buf = mems[1].tobuf()

In [ ]:
obj = mems[1]

In [ ]:
obj.get_info()

In [ ]:
import io
import os

In [ ]:
dest_path = 'build_cache/nocereal/tuplex-package.tar.gz'
src_path = 'build_cache/nocereal/tuplex.tar'
lambda_src_path = 'build_cache/nocereal/lambda-runner.tar'


allowed_prefixes = ['python/tuplex', 'python/setup.py', 'python/MANIFEST.in']

dest_root = ''

with tarfile.open(src_path, 'r') as tf:
    
    mems = tf.getmembers()

    with tarfile.open(dest_path, 'w:gz') as target:
        for obj in mems:
            
            if len(list(filter(lambda prefix: obj.name.startswith(prefix), allowed_prefixes))) > 0:
                if obj.name.endswith('tar.gz'):
                    continue

                if os.path.splitext(obj.name)[1][1:] not in {'in', 'py', 'so', 'dylib', 'toml'}:
                    continue
                fileobj = tf.extractfile(obj)    
                obj.name = obj.name.replace('python/', dest_root)
                target.addfile(obj, fileobj)
                print('adding {}'.format(obj.name))
        if lambda_src_path is not None:
            with tarfile.open(lambda_src_path, 'r') as tf_lam:
                obj = list(filter(lambda x: x.name.endswith('tplxlam.zip'), tf_lam.getmembers()))[0]
                fileobj = tf_lam.extractfile(obj)    
                obj.name = os.path.join(dest_root, 'tuplex/other/tplxlam.zip')
                target.addfile(obj, fileobj)
                print('adding {}'.format(obj.name))

In [ ]:
import boto3

In [ ]:
iam= boto3.client('iam')

In [ ]:
lambda_role='tuplex-lambda-role'
policy_names = iam.list_role_policies(RoleName=lambda_role)['PolicyNames']

# arn:aws:iam::"${account_id}":policy/"${policy_name}"

In [ ]:
def get_policy_arn(policy_name):
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    account_id = identity['Account']
    
    return 'arn:aws:iam::{}:policy/{}'.format(account_id, policy_name)

In [ ]:
arns = [get_policy_arn(name) for name in policy_names]

In [ ]:
policy_names

In [ ]:
iam.detach_role_policy(RoleName=lambda_role, PolicyArn=arns[1])

In [ ]:
s3 = boto3.client('s3')

In [ ]:
s3.delete_object(Bucket = 'tuplex-tuplex', Key='scratch/lambda-deploy.zip')